---
title: Generator and Decorator
abstract: |
    This notebook delves deeper into functional programming techniques, covering the syntax for generators and the design patterns for decorators. Along the way, readers will also explore how to specify positional and keyword arguments in function definitions, as well as how to use default arguments and handle variable numbers of arguments. Finally, we’ll demonstrate how to organize various functions into a module or a package containing multiple submodules.
---

In [ ]:
from __init__ import install_dependencies

await install_dependencies()

In [ ]:
%load_ext divewidgets
%load_ext jupyter_ai
%ai update chatgpt dive:chat

## Generator

Recall the implementation of Fibonacci numbers using an interation:

In [ ]:
def fibonacci_iteration(n):
    if n > 1:
        _, F = 0, 1  # next two Fibonacci numbers
        while n > 1:
            _, F, n = F, F + _, n - 1
        return F
    elif n == 1:
        return 1
    else:
        return 0


fibonacci_iteration(10)

The code can be modified to generate a Fibonacci sequence efficiently:

In [ ]:
%%optlite -l -h 1000
def create_fibonacci(Fn, Fnn):
    def next():
        """Returns the next (generalized) Fibonacci number starting with
        Fn and Fnn as the first two numbers."""
        nonlocal Fn, Fnn, n
        value = Fn
        Fn, Fnn, n = Fnn, Fn + Fnn, n + 1
        return value

    def self():  # make the return object callable to replace print_fibonacci_state
        print(
            """States:
        Next Fibonacci number      = {}
        Next next Fibonacci number = {}
        Next order                 = {}""".format(
                Fn, Fnn, n
            )
        )

    n = 0

    self.next = next  # add next as an attribute of self
    return self       # to be returned


fib = create_fibonacci(0, 1)
n = 0
while n < 5:
    print(fib.next())
    n += 1
fib()

# For loop fails. Why?
fib = create_fibonacci(0, 1)
for i in fib:
    print(i)

Unfortunately, the Fibonacci object is not an iterable so one cannot directly iterate over it using a for loop. It is also cubersome to have to write nested functions.

Python provides an easy way to create an iterator that generates a sequence of objects:

In [ ]:
fibonacci_generator = (fibonacci_iteration(n) for n in range(3))
fibonacci_generator

The above uses a [*generator expression*](https://docs.python.org/3/reference/expressions.html#grammar-token-generator-expression) to define the *generator* object `fibonacci_generator`.

Since a generator is an iterator, we can use the [`next` function](https://docs.python.org/3/library/functions.html#next) to obtain the next item.

In [ ]:
fibonacci_generator = (fibonacci_iteration(n) for n in range(3))

while True:
    print(next(fibonacci_generator))  # raises StopIterationException eventually

We can also use a `for` loop to handle the exception:

In [ ]:
fibonacci_generator = (fibonacci_iteration(n) for n in range(5))

for fib in fibonacci_generator:  # StopIterationException handled by for loop
    print(fib)

The implementation of `fibonacci_generator` is not efficient because of redundant computations. In order to store the last two computed Fibonacci numbers, a better way to is create local states like defining a function. This can be done using the keyword [`yield`](https://docs.python.org/3/reference/expressions.html?highlight=yield#yield-expressions):

In [ ]:
%%optlite -h 450
def fibonacci_sequence(Fn, Fnn, stop):
    """Return a generator that generates Fibonacci numbers
    starting from Fn and Fnn until stop (exclusive)."""
    while Fn < stop:
        yield Fn  # return Fn and pause execution
        Fn, Fnn = Fnn, Fnn + Fn


for fib in fibonacci_sequence(0, 1, 5):
    print(fib)

::::{important} How does `yield` work?

1. Invoking a function that is defined with the `yield` keyword returns a *generator* without executing the function body.
2. Calling `next` on the generator initiates or resumes execution, which:
    - Pauses at the next `yield` expression, if present, or
    - Raises a `StopIterationException` when the execution completes.

::::

::::{exercise}
:label: ex:return-yield

As a test of your understanding, explain what gets printed by the following code.

::::

In [ ]:
def f():
    return 0
    yield 1


for i in f():
    print(i)

YOUR ANSWER HERE

::::{exercise}
:label: ex:doc 

`yield` can be both a statement and an expression. As an expression, it can be used to receive values sent to the generator: 

- The value of a `yield` expression is `None` by default, but 
- it can be set by the `generator.send` method.

Add the document string to the following function. In particular, explain the effect of calling the method `send` on the returned generator.

::::

In [ ]:
def fibonacci_sequence(Fn, Fnn, stop):
    # YOUR CODE HERE
    raise NotImplementedError
    while Fn < stop:
        value = yield Fn
        if value is not None:
            Fnn = value  # set next number to the value of yield expression
        Fn, Fnn = Fnn, Fnn + Fn


fibonacci_generator = fibonacci_sequence(0, 1, 5)
print(next(fibonacci_generator))
print(fibonacci_generator.send(2))
for fib in fibonacci_generator:
    print(fib)

## Optional Arguments

Fibonacci sequence normally starts with `0` and `1` by default. Is it possible to make arguments `Fn` and `Fnn` optional with default values?

**How to give arguments default values?**

In [ ]:
def fibonacci_sequence(Fn=0, Fnn=1, *, stop=None):
    while stop is None or Fn < stop:
        value = yield Fn
        Fn, Fnn = Fnn, Fnn + Fn

Arguments with default values specified by `=...` are called [default arguments](https://docs.python.org/3/tutorial/controlflow.html#default-argument-values). They are optional in the function call.

In [ ]:
for fib in fibonacci_sequence(stop=5):
    print(fib)  # with default Fn=0, Fnn=1

`stop=5` in the function call is called a [keyword argument](https://docs.python.org/3/glossary.html#term-keyword-argument). As supposed to [*positional arguments*](https://docs.python.org/3/glossary.html#term-argument), it specifies the name of the argument explicitly. Indeed, `stop` is a [keyword-only argument](https://peps.python.org/pep-3102/), which can not be specified as a positional argument:

In [ ]:
for fib in fibonacci_sequence(0, 1, 5):
    print(fib)

::::{exercise}
:label: ex:fib1

Is `fibonacci_sequence(stop=5)` the same as `fibonacci_sequence(5)`? In particular, what is the behavior of the following code?

::::

In [ ]:
for fib in fibonacci_sequence(5):
    print(fib)
    if fib > 10:
        break  # Will this be executed?

YOUR ANSWER HERE

::::{important} Rules for specifying arguments

1. Default (Keyword) arguments must be after all non-default (positional) arguments in a function definition (call).
1. The value for an argument cannot be specified more than once in a function definition (call).
::::

E.g., the following results in an error:

In [ ]:
fibonacci_sequence(stop=10, 1)

In [ ]:
fibonacci_sequence(1, Fn=1)

The following shows that the behavior of `range` is different.

In [ ]:
for count in range(1, 10, 2):
    print(count, end=" ")  # counts from 1 to 10 in steps of 2
print()
for count in range(1, 10):
    print(count, end=" ")  # default step=1
print()
for count in range(10):
    print(count, end=" ")  # default start=0, step=1
range(stop=10)  # fails

`range` takes only positional arguments.  
However, the first positional argument has different intepretations (`start` or `stop`) depending on the number of arguments (2 or 1).

`range` is indeed NOT a generator. How is range implemented?

In [ ]:
print(type(range), type(range(10)))

## Variable number of arguments

[The implementation of range](https://github.com/python/cpython/blob/6afb285ff0790471a6858e44f85d143f07fda70c/Objects/rangeobject.c#L82-L123) uses a [variable number of arguments](https://docs.python.org/3.4/tutorial/controlflow.html#arbitrary-argument-lists).

In [ ]:
def print_arguments(*args, **kwargs):
    """Take any number of arguments and prints them"""
    print("args ({}): {}".format(type(args), args))
    print("kwargs ({}): {}".format(type(kwargs), kwargs))


print_arguments(0, 10, 2, start=1, stop=2)

- `args` is a tuple of positional arguments.
- `kwargs` is a dictionary of keyword arguments, which is a list of values indexed by unique keys that are not necessary integers.

In [ ]:
d = {'start': 1, 'stop': 2}
d['start'], d['stop'], d.keys(), d.values(), d.items()

`*` and `**` are *unpacking operators* for tuple/list and dictionary respectively:

In [ ]:
args = (0, 10, 2)
kwargs = {"start": 1, "stop": 2}
print_arguments(*args, **kwargs)

The following function converts all the arguments to a string, which will be useful later on.

In [ ]:
def argument_string(*args, **kwargs):
    """Return the string representation of the list of arguments."""
    return "({})".format(
        ', '.join(
            [
                *[f'{v!r}' for v in args],  # arguments
                *[
                    f'{k}={v!r}' for k, v in kwargs.items()
                ],  # keyword arguments
            ]
        )
    )


argument_string(0, 10, 2, start=1, stop=2)

::::{seealso} Representation
 `!r` convert `v` to the string representation (`repr`) that can be evaluated by python `eval`. In particular, `'a'` will be converted to `"'a'"`, which has the quotation needed for the string literal. See [token conversion](https://docs.python.org/3/reference/lexical_analysis.html#grammar-token-conversion).
::::

In [ ]:
repr?
print(repr('a'))
repr('a')

::::{exercise}
:label: ex:fib2

Redefine `fibonacci_sequence` so that the positional arguments depend on the number of arguments:

::::

In [ ]:
def fibonacci_sequence(*args):
    """Return a generator that generates Fibonacci numbers
    starting from Fn and Fnn to stop (exclusive).
    generator.send(value) sets next number to value.

    fibonacci_sequence(stop)
    fibonacci_sequence(Fn,Fnn)
    fibonacci_sequence(Fn,Fnn,stop)
    """
    Fn, Fnn, stop = 0, 1, None  # default values

    # handle different number of arguments
    if len(args) == 1:
        # YOUR CODE HERE
        raise NotImplementedError
    elif len(args) == 2:
        Fn, Fnn = args[0], args[1]
    elif len(args) > 2:
        Fn, Fnn, stop = args[0], args[1], args[2]

    while stop is None or Fn < stop:
        value = yield Fn
        if value is not None:
            Fnn = value  # set next number to the value of yield expression
        Fn, Fnn = Fnn, Fnn + Fn

In [ ]:
for fib in fibonacci_sequence(5):  # default Fn=0, Fn=1
    print(fib)

In [ ]:
for fib in fibonacci_sequence(1, 2):  # default stop=None
    print(fib)
    if fib > 5:
        break

In [ ]:
args = (1, 2, 5)
for fib in fibonacci_sequence(*args):  # default stop=None
    print(fib)

## Decorator

The code below decorates the `fibonacci` function by printing each recursive call and the depth of the call stack.

In [ ]:
def fibonacci(n):
    """Returns the Fibonacci number of order n."""
    global count, depth
    count += 1
    depth += 1
    print("{:>3}: {}fibonacci({!r})".format(count, "|" * depth, n))

    value = fibonacci(n - 1) + fibonacci(n - 2) if n > 1 else 1 if n == 1 else 0

    depth -= 1
    if depth == -1:  # recursion done
        print("Done")
        count = 0  # reset count for subsequent recursions
    return value


count, depth = 0, -1
for n in range(6):
    print(fibonacci(n))

The decoration is useful in showing the efficiency of the function, but it rewrites the function definition.

**How to decorate a function without changing its implementation?**

Decorations are often temporary. Is it possible to avoid

- going through the source codes to remove decorations?
- switching back and forth between the original and decorated codes?

::::{admonition} Attempt
What about defining a new function that calls and decorates the original function?
::::

In [ ]:
def fibonacci(n):
    """Returns the Fibonacci number of order n."""
    return fibonacci(n - 1) + fibonacci(n - 2) if n > 1 else 1 if n == 1 else 0


def fibonacci_decorated(n):
    """Returns the Fibonacci number of order n."""
    global count, depth
    count += 1
    depth += 1
    print("{:>3}: {}fibonacci({!r})".format(count, "|" * depth, n))

    value = fibonacci(n)

    depth -= 1
    if depth == -1:  # recursion done
        print("Done")
        count = 0  # reset count for subsequent recursions
    return value


count, depth = 0, -1
for n in range(6):
    print(fibonacci_decorated(n))

::::{exercise}
:label: ex:fib3

Explain whether the attempt works.

::::

YOUR ANSWER HERE

````{admonition} Attempt
What about renaming `fibonacci_decorated` to `fibonacci`?

```python
fibonacci = fibonacci_decorated
count, depth = 0, -1
fibonacci_decorated(10)
```

(If you are faint-hearted, don't run the above code.)
````

::::{exercise}
:label: ex:fib4 

Explain whether the attempt works.

::::

YOUR ANSWER HERE

An elegant solution is to

- capture the function to be decorated in the closure of the decorated function, and
- rename the decorated function to the same name as the function to be decorated.

In [ ]:
def print_function_call(f):
    def wrapper(*args, **kwargs):
        nonlocal count, depth
        count += 1
        depth += 1
        call = f"{f.__name__}{argument_string(*args, **kwargs)}"
        print(f"{count:>3}:{'|' * depth}{call}")
        value = f(*args, **kwargs)  # calls f
        depth -= 1
        if depth == -1:
            print("Done")
            count = 0
        return value

    count, depth = 0, -1
    return wrapper  # return the decorated function

The above defines a *decorator* `print_function_call` that takes in a function `f` to be decorated and returns the decorated function `wrapper` that captures and decorates `f`:
- `wrapper` expects the same set of arguments for `f`,  
- returns the same value returned by `f` on the arguments, but
- can execute additional codes before and after calling `f` to print the function call.

By redefining `fibonacci` as the returned `wrapper`, the original `fibonacci` captured by `wrapper` calls `wrapper` as desired.

In [ ]:
def fibonacci(n):
    """Returns the Fibonacci number of order n."""
    return fibonacci(n - 1) + fibonacci(n - 2) if n > 1 else 1 if n == 1 else 0


fibonacci = print_function_call(fibonacci)  # so original fibonnacci calls wrapper
fibonacci(5)

The redefinition does not change the original `fibonacci` captured by `wrapper`.

In [ ]:
import inspect

for cell in fibonacci.__closure__:
    if callable(cell.cell_contents):
        print(inspect.getsource(cell.cell_contents))

Python provides the [*syntactic sugar*](https://en.wikipedia.org/wiki/Syntactic_sugar) below to simplify the redefinition.

In [ ]:
@print_function_call
def fibonacci(n):
    """Returns the Fibonacci number of order n."""
    return fibonacci(n - 1) + fibonacci(n - 2) if n > 1 else 1 if n == 1 else 0


fibonacci(5)

::::{exercise}
:label: ex:fib5

Why use a variable number of arguments in `wrapper`? You may use `optlite` below to visualize the execution step-by-step.

::::

YOUR ANSWER HERE

::::{seealso} Design patterns

The decorator is one of many software [design patterns](https://en.wikipedia.org/wiki/Software_design_pattern) that can be reused across various scenarios and applications. Design patterns are not reusable code but rather *reusable methods for writing good code*. They differ from algorithms, which are methods of computation with specific running times, as design patterns do not have running times associated with them.

::::

In [ ]:
%%ai chatgpt -f text
Explain in one paragraph what software design pattern is and why decorator is considered a design pattern but generator is not.

## Examples of Decorators

Note that the decorated `fibonacci` does not have the correct docstring. Even the function name is wrong.

In [ ]:
help(fibonacci)

This can be fixed using decorator `@functools.wraps`:

In [ ]:
import functools

def print_function_call(f):
    @functools.wraps(f)  # give wrapper the identity of f and more
    def wrapper(*args, **kwargs):
        nonlocal count, depth
        count += 1
        depth += 1
        call = "{}{}".format(f.__name__, "({})".format(", ".join([*(f"{v!r}" for v in args), *(f"{k}={v!r}" for k, v in kwargs.items())])))
        print(f"{count:>3}:{'|' * depth}{call}")
        value = f(*args, **kwargs)  # calls f
        depth -= 1
        if depth == -1:
            print("Done")
            count = 0
        return value

    count, depth = 0, -1
    return wrapper  # return the decorated function

@print_function_call
def fibonacci(n):
    """Returns the Fibonacci number of order n."""
    return fibonacci(n - 1) + fibonacci(n - 2) if n > 1 else 1 if n == 1 else 0

fibonacci(5)
help(fibonacci)

::::{note} What does `@functools.wraps(f)` do?

The decoration `@functools.wraps(f)`

- makes some attributes (such as `__name__`, `__module__`, `__doc__`, ...) of the decorated function the same as those of the original function, and
- adds some useful attributes such as `__wrapped__` that points to the original function.

::::

We can also undo the decoration using `__wrapped__`.

In [ ]:
fibonacci, fibonacci_decorated = fibonacci.__wrapped__, fibonacci  # recover
print("original fibonacci:")
print(fibonacci(5))

fibonacci = fibonacci_decorated  # decorate
print("decorated fibonacci:")
print(fibonacci(5))

Another application is to use decorator to improve recursions. For instance, we can make recursion more efficient by caching the return values.

In [ ]:
def caching(f):
    """Cache the return value of a function that takes a single argument.

    Parameters
    ----------
    f: Callable
        A function that takes a single argument.

    Returns
    -------
    Callable:
        The function same as f but has its return valued automatically cached
        when called. It has a method cache_clear to clear its cache.
    """

    @functools.wraps(f)
    def wrapper(n):
        if n not in cache:
            cache[n] = f(n)
        else:
            print("read from cache")
        return cache[n]

    cache = {}
    wrapper.cache_clear = lambda: cache.clear()  # add method to clear cache
    return wrapper


@print_function_call
@caching
def fibonacci(n):
    """Returns the Fibonacci number of order n."""
    return fibonacci(n - 1) + fibonacci(n - 2) if n > 1 else 1 if n == 1 else 0

fibonacci(5)
fibonacci(5)
fibonacci.cache_clear()
fibonacci(5)

`cache` is a [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries), which will be formally introduced later in the course. For now, think of `cache[n]` as a variable that stores the computed value of `F_n` to avoid redundant calculations. If you are curious about why a dictionary is used, consider exploring the following question:

In [ ]:
%%ai chatgpt -f text
Explain in one paragraph why one would use a Python dictionary instead of a list to store values with integer keys.

`functools` also provides a similar decorator called `lru_cache`, which can be applied to functions with multiple input arguments. It also allows you to specify a maximum cache size with a default value of 128. This means that the *least recently used (lru)* items are automatically removed from the cache when the cache size reaches its limit.

In [ ]:
@print_function_call
@functools.lru_cache
def fibonacci(n):
    """Returns the Fibonacci number of order n."""
    return fibonacci(n - 1) + fibonacci(n - 2) if n > 1 else 1 if n == 1 else 0

fibonacci(5)
fibonacci(5)

To clear the cache, we can use the `cache_clear` method added by `@functools.lru_cache`:

In [ ]:
fibonacci.__wrapped__.cache_clear()
fibonacci(5)

Note that `fibonacci.cache_clear()` results in an error unless we call `update_wrapper` first as follows. (Why?)

In [ ]:
functools.update_wrapper(fibonacci, fibonacci.__wrapped__, assigned=('cache_clear',))
fibonacci.cache_clear()
fibonacci(5)

In [ ]:
%%ai chatgpt -f text
Explain in one paragraph what LRU means in caching and why a strategy like LRU is used.

## Writing a Module

**How to create a module?**

To create a module, simply put the code in a python source file `<module name>.py` in
- the current directory, or
- a python *site-packages* directory in system path.

In [ ]:
import sys

print(sys.path)

For example, `recurtools.py` in the current directory defines the module `recurtools`.

In [ ]:
from IPython.display import Code

Code(filename="recurtools.py", language="python")

The module provides the decorators `print_function_call` and `caching` defined earlier.

In [ ]:
import recurtools as rc


@rc.print_function_call
@rc.caching
def factorial(n):
    return factorial(n - 1) if n > 1 else 1

In [ ]:
factorial(5)
factorial(5)
factorial.cache_clear()
factorial(5)

In Python, large modules often consist of many submodules, which can themselves contain further submodules. To manage this complexity, Python uses packages. A package is essentially a directory that contains an `__init__.py` file, which serves to initialize the package. For instance, if we go up one directory level, `Lecture6` becomes a package on the search path that contains the `recurtools` as a submodule:

In [ ]:
%%bash
cd .. && python -c 'from Lecture6.recurtools import *; help(print_function_call)'

A submodule can [import another submodule relatively](https://peps.python.org/pep-0328/#id3). It can also [run as a script](https://docs.python.org/3/library/__main__.html#name-main).

In [ ]:
Code(filename="demo.py", language="python")

In [ ]:
%%bash
cd .. && python -m 'Lecture6.demo'

In [ ]:
%%ai chatgpt -f text
How to use Sphinx and its AutoDoc extension to create a user manual for a package?